In [1]:
import coiled

import fsspec
import numpy as np
import rioxarray
import xarray as xr
import fsspec
import pandas as pd
import logging
from flox.xarray import xarray_reduce
import numpy as np

import dask
import zarr
import gcsfs
import glob


In [2]:
commodities = glob.glob("/Users/justin.terry/Downloads/spam2020V1r0_global_physical_area/*A.tif")

In [10]:
data_arrs = {commodity[-10:-6]:xr.open_dataset(commodity).band_data for commodity in commodities}
data_arrs

{'VEGE': <xarray.DataArray 'band_data' (band: 1, y: 2160, x: 4320)> Size: 37MB
 [9331200 values with dtype=float32]
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 35kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
   * y            (y) float64 17kB 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
     spatial_ref  int64 8B ...
 Attributes:
     AREA_OR_POINT:  Area,
 'REST': <xarray.DataArray 'band_data' (band: 1, y: 2160, x: 4320)> Size: 37MB
 [9331200 values with dtype=float32]
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 35kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
   * y            (y) float64 17kB 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
     spatial_ref  int64 8B ...
 Attributes:
     AREA_OR_POINT:  Area,
 'SUGC': <xarray.DataArray 'band_data' (band: 1, y: 2160, x: 4320)> Size: 37MB
 [9331200 values with dtype=float32]
 Coordinates:
   * band         (band) int64 8B 1
   * x            (x) float64 35kB -180.0 -179.9

In [24]:
presence = xr.Dataset({})
for k, v in data_arrs.items():
    presence[k] = ~np.isnan(v).astype(bool)

# for some reason I need to apply this twice to get all the data variables as bools?
for data_var in list(dataset.data_vars):
    presence[data_var] = ~np.isnan(presence[data_var]).astype(bool)

presence

<xarray.Dataset> Size: 429MB
Dimensions:      (band: 1, x: 4320, y: 2160)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 35kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * y            (y) float64 17kB 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
    spatial_ref  int64 8B ...
Data variables: (12/46)
    VEGE         (band, y, x) bool 9MB True True True True ... True True True
    REST         (band, y, x) bool 9MB True True True True ... True True True
    SUGC         (band, y, x) bool 9MB False False False ... False False False
    SORG         (band, y, x) bool 9MB False False False ... False False False
    MAIZ         (band, y, x) bool 9MB False False False ... False False False
    BANA         (band, y, x) bool 9MB True True True True ... True True True
    ...           ...
    PMIL         (band, y, x) bool 9MB False False False ... False False False
    COWP         (band, y, x) bool 9MB False False False ... False False False
    TROF         (band, y, x) bool 9MB True True True True ... True True True
    CITR         (band, y, x) bool 9MB True True True True ... True True True
    YAMS         (band, y, x) bool 9MB True True True True ... True True True
    BEAN         (band, y, x) bool 9MB False False False ... False False False

In [25]:
presence.to_zarr("mapspam_2020_presence_all.zarr")

/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/site-packages/zarr/api/asynchronous.py:197: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [2]:
cluster = coiled.Cluster(
    name="dist_alerts_zonal_stat_count",
    region="us-east-1",
    n_workers=5,
    tags={"project": "dist_alerts_zonal_stat"},
    scheduler_vm_types="r7g.xlarge",
    worker_vm_types="r7g.2xlarge",
    compute_purchase_option="spot_with_fallback"
)

client = cluster.get_client()
cluster.adapt(minimum=5, maximum=75)

Output()

ERROR:root:code for hash blake2b was not found.
Traceback (most recent call last):
  File "/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/hashlib.py", line 245, in <module>
    globals()[__func_name] = __get_hash(__func_name)
                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/hashlib.py", line 129, in __get_openssl_constructor
    return __get_builtin_constructor(name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/hashlib.py", line 123, in __get_builtin_constructor
    raise ValueError('unsupported hash type ' + name)
ValueError: unsupported hash type blake2b
ERROR:root:code for hash blake2s was not found.
Traceback (most recent call last):
  File "/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/hashlib.py", line 245, in <module>
    globals()[__func_name] = __get_hash(__func_name)
                             ^^^^^^^^^^^^^^^^^^^^^^^
  

╭──────────────────────────────── Package Info ────────────────────────────────╮
│             ╷                                                                │
│   Package   │ Note                                                           │
│ ╶───────────┼──────────────────────────────────────────────────────────────╴ │
│   python    │ https://conda.anaconda.org/conda-forge/,conda-forge            │
│             ╵                                                                │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

INFO:distributed.deploy.adaptive:Adaptive scaling started: minimum=5 maximum=75


In [3]:
countries = xr.open_zarr(
    's3://gfw-data-lake/gadm_administrative_boundaries/v4.1.85/raster/epsg-4326/zarr/adm0_clipped_to_dist.zarr'
).band_data.coarsen(x=333, y=222, boundary="trim").median().compute()

In [6]:
countries.to_zarr("resampled_countries_mapspam.zarr")

/Users/justin.terry/.pyenv/versions/3.12.9/lib/python3.12/site-packages/zarr/api/asynchronous.py:197: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [7]:
cluster.shutdown()

INFO:distributed.deploy.adaptive:Adaptive scaling stopped: minimum=5 maximum=75. Reason: unknown


In [8]:
presence = xr.open_dataset("mapspam_2020_presence_all.zarr")

In [9]:
presence

<xarray.Dataset> Size: 429MB
Dimensions:      (band: 1, y: 2160, x: 4320)
Coordinates:
    spatial_ref  int64 8B ...
  * band         (band) int64 8B 1
  * x            (x) float64 35kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * y            (y) float64 17kB 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
Data variables: (12/46)
    BARL         (band, y, x) bool 9MB ...
    TEAS         (band, y, x) bool 9MB ...
    CNUT         (band, y, x) bool 9MB ...
    OCER         (band, y, x) bool 9MB ...
    RCOF         (band, y, x) bool 9MB ...
    COFF         (band, y, x) bool 9MB ...
    ...           ...
    SESA         (band, y, x) bool 9MB ...
    SUNF         (band, y, x) bool 9MB ...
    BANA         (band, y, x) bool 9MB ...
    GROU         (band, y, x) bool 9MB ...
    REST         (band, y, x) bool 9MB ...
    PMIL         (band, y, x) bool 9MB ...

In [30]:
countries_matched = countries.rio.reproject_match(presence.SUGC)

In [62]:
results = xarray_reduce(
    countries_matched,
    *(countries_matched, presence.SUGC, presence.VEGE),
    func='count',
    # expected_groups=(
    #     adm0_ids,
    #     np.arange(86),
    #     np.arange(854),
    #     np.arange(22),
    #     alert_dates,
    #     [1, 2, 3]
    # ),
    # reindex=ReindexStrategy(
    #     blockwise=False, array_type=ReindexArrayType.SPARSE_COO
    # ),
    # fill_value=0
)
results

<xarray.DataArray 'band_data' (band_data: 223, SUGC: 1, VEGE: 1)> Size: 2kB
array([[[4556418]],

       [[   9058]],

       [[    446]],

       [[  30688]],

       [[      2]],

       [[      7]],

       [[  14919]],

       [[      5]],

       [[   1312]],

       [[  40065]],

...

       [[  11069]],

       [[ 126226]],

       [[   3281]],

       [[   2428]],

       [[   6990]],

       [[  10777]],

       [[      2]],

       [[     30]],

       [[   5484]],

       [[   8999]]])
Coordinates:
    spatial_ref  int64 8B 0
  * band_data    (band_data) float64 2kB 0.0 4.0 8.0 12.0 ... 882.0 887.0 894.0
  * SUGC         (SUGC) bool 1B False
  * VEGE         (VEGE) bool 1B True
Attributes:
    AREA_OR_POINT:             Area
    STATISTICS_MAXIMUM:        124
    STATISTICS_MEAN:           124
    STATISTICS_MINIMUM:        124
    STATISTICS_STDDEV:         0
    STATISTICS_VALID_PERCENT:  0.0116
    _FillValue:                nan

In [60]:
dim_names = results.dims
indices = results.coords
values = results.data

coord_dict = {
    dim: results.coords[dim].values[indices[i]]
    for i, dim in enumerate(dim_names)
}
coord_dict["value"] = values

df = pd.DataFrame(coord_dict)

KeyError: 0

In [56]:
dim_names

('band_data', 'SUGC')

In [74]:
presence["country"] = countries_matched

In [92]:
results = presence.squeeze("band").to_dataframe().reset_index().drop("x", axis=1).drop("y", axis=1).groupby(["country"]).any()

In [93]:
results

,BARL,TEAS,CNUT,OCER,RCOF,COFF,TOBA,SORG,COTT,CASS,...,VEGE,SWPO,RUBB,PLNT,SESA,SUNF,BANA,GROU,REST,PMIL
country,,,,,,,,,,,,,,,,,,,,,
0.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
4.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
8.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
12.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
16.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
876.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False
882.0,False,False,True,False,False,False,False,False,False,True,...,True,True,False,True,False,False,True,False,True,False


In [91]:
presence.OCER

<xarray.DataArray 'OCER' (band: 1, y: 2160, x: 4320)> Size: 9MB
array([[[False, False, ..., False, False],
        [False, False, ..., False, False],
        ...,
        [False, False, ..., False, False],
        [False, False, ..., False, False]]], shape=(1, 2160, 4320))
Coordinates:
    spatial_ref  int64 8B 0
  * band         (band) int64 8B 1
  * x            (x) float64 35kB -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * y            (y) float64 17kB 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
Attributes:
    AREA_OR_POINT:  Area